# Agentes con Transformers

*Este notebook es una adaptación de:* https://huggingface.co/docs/transformers/main/en/agents

Los modelos de lenguaje tienen una indudable capacidad para intercambiar información a través del lenguaje natural. Sin embargo, a veces muestran una sorprendente incapacidad para realizar tareas sencillas, especialmente en ciertos dominios como el razonamiento lógico, los cálculos matemáticos o la búsqueda de información.

Una aproximación para superar esta limitación es el uso de *agentes*. Un agente es un sistema basado en el uso de un modelo de lenguaje como motor, y que además está equipado con acceso a ciertas herramientas (*tools*) que puede emplear para realizar tareas muy específicas. Por ejemplo, un agente podría estar preparado para que cuando interprete que alguien le pide que realice un cálculo matemático, utilizar algún tipo de software matemático para realizarlo.

Los agente pueden ser programados para:

- Concebir una serie de acciones/herramientas y ejecutarlas todas a la vez, como el [CodeAgent](https://huggingface.co/docs/transformers/main/en/main_classes/agent#transformers.CodeAgent)
- Planear y luego ejecutar una serie de acciones (una a una) esperando a que cada una se complete antes de continua con la siguiente, como el [ReactJsonAgent](https://huggingface.co/docs/transformers/main/en/main_classes/agent#transformers.ReactJsonAgent)

## Tipos de agentes

### Agentes Code

Este agente tiene un paso de planificación que genera código Python para ejecutar todas sus acciones de una vez. Nativamente es capaz de manejar diferentes tipos de entradas y salidas, por lo tanto es la opción recomendada para tareas multimodales.

### Agentes React

Este es el agente recomendado para resolver tareas de razonamiento, ya que el framework React es eficiente para pensar paso a paso, y cada paso usando las salidas del paso anterior. Transformers implementa dos versiones del *ReactJsonAgent*:

- *ReactJsonAgent*: genera llamadas a herramientas en forma de JSON en su salida.
- *ReactCodeAgente*: e un nuevo tipo de *ReactJsonAgent* que genera sus llamadas a herramientas como *blobs* (binarios ejectuables) de código.
